<a href="https://colab.research.google.com/github/OdysseusPolymetis/colabs_for_nlp/blob/main/4_vecteurs_simples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stanza

Pour cette partie-là, il vous faut créer un dossier "dossier" et y mettre vos txt.

In [ ]:
import os
import stanza
from gensim.models import Word2Vec
import numpy as np

stanza.download('fr')
nlp = stanza.Pipeline('fr', processors='tokenize,pos,lemma')

folder_path = "/content/dossier"

define_batch_size = 1024  # Taille dynamique du batch

def batch_process_texts(folder_path, batch_size=define_batch_size):
    all_sentences = []
    texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())

    paragraphs = "\n".join(texts).split("\n")
    batches = [paragraphs[i:i + batch_size] for i in range(0, len(paragraphs), batch_size)]

    for batch in batches:
        batch_text = "\n".join(batch)
        doc = nlp(batch_text)
        for sentence in doc.sentences:
            lemmatized_sentence = [word.lemma for word in sentence.words if word.lemma is not None]
            all_sentences.append(lemmatized_sentence)

    return all_sentences

sentences = batch_process_texts(folder_path, batch_size=define_batch_size)

model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, sg=1)
model.save("word2vec_model.model")

def word_similarity(word1, word2):
    return model.wv.similarity(word1, word2)

def word_analogy(word1, word2, word3):
    return model.wv.most_similar(positive=[word2, word3], negative=[word1])

In [8]:
def most_similar_words(word, topn=10):
    try:
        similar_words = model.wv.most_similar(word, topn=topn)
        for w, sim in similar_words:
            print(f"{w}: {sim:.4f}")
    except KeyError:
        print(f"Le mot '{word}' n'est pas dans le vocabulaire du modèle.")

In [5]:
with open("vectors.tsv", "w", encoding="utf-8") as vec_file, open("metadata.tsv", "w", encoding="utf-8") as meta_file:
    for word in model.wv.index_to_key:
        vector = model.wv[word]
        vec_file.write("\t".join([str(x) for x in vector]) + "\n")
        meta_file.write(word + "\n")

print("Export terminé : vectors.tsv et metadata.tsv sont prêts pour TensorFlow Projector.")

Export terminé : vectors.tsv et metadata.tsv sont prêts pour TensorFlow Projector.


In [7]:
word_similarity("esprit", "courage")

0.88316154

In [13]:
most_similar_words("amour", topn=10)

créature: 0.9343
haine: 0.9331
bonheur: 0.9309
cause: 0.9306
vengeance: 0.9290
amoureux: 0.9274
prouver: 0.9225
démon: 0.9206
histoire: 0.9170
honte: 0.9165
